### Integração Numérica - Regra de 1/3 de Simpson

Este notebook apresenta a implementação da Regra de 1/3 de Simpson para o cálculo de integrais definidas, com cálculo do erro de aproximação simples e generalizado, utilizando orientação a objetos com uma classe base comum.


In [2]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Classe `IntegralUmTercoSimpson`

In [6]:
class IntegralUmTercoSimpson(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None, func_integral=None, h=None):
        super().__init__()
        self._setup(x, y, limite_inf, limite_sup, func_integral, h)

        # Ajustar se pontos não múltiplos de 2 (n deve ser par)
        if (len(self.x)) % 2 != 0:
            raise ValueError("Para Simpson 1/3, número de intervalos deve ser par (n deve ser múltiplo de 2).")

    def integral(self):
        n = len(self.x) - 1
        soma = self.y[0] + self.y[-1]

        for i in range(1, n):
            if i % 2 == 0:
                soma += 2 * self.y[i]
            else:
                soma += 4 * self.y[i]

        return (self.h / 3) * soma

    def erro_simples(self):
        d4_inf = self.derivada_quarta_ordem(self.limite_inf)
        d4_sup = self.derivada_quarta_ordem(self.limite_sup)
        maior = self.saber_maior(d4_inf, d4_sup)
        parcela_1 = ((self.h ** 4) * (self.limite_sup - self.limite_inf)) / 180
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        return self.erro_simples()

### Exemplo de uso com pontos

In [7]:
# Célula 3 - Integral Simpson 1/3 com pontos (x, y)
pontos_x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
def f(x):
    return x * math.sqrt(x**2 + 1)
pontos_y = [f(xi) for xi in pontos_x]

simpson_13 = IntegralUmTercoSimpson(x=pontos_x, y=pontos_y)
print("Simpson 1/3 (x, y):", simpson_13.integral())
print("Erro simples:", simpson_13.erro_simples())
print("Erro generalizado:", simpson_13.erro_generalizado())


Simpson 1/3 (x, y): 0.11611322435059976
Erro simples: 1.2970517815259231e-06
Erro generalizado: 1.2970517815259231e-06


### Exemplo de uso com função

In [8]:
# Célula 4 - Integral Simpson 1/3 com função, limites e h
def f(x):
    return x * math.sqrt(x**2 + 1)

simpson_13_2 = IntegralUmTercoSimpson(limite_inf=0, limite_sup=0.5, func_integral=f, h=0.1)
print("Simpson 1/3 (função):", simpson_13_2.integral())
print("Erro simples:", simpson_13_2.erro_simples())
print("Erro generalizado:", simpson_13_2.erro_generalizado())


Simpson 1/3 (função): 0.11611322435059977
Erro simples: 1.2970517815262803e-06
Erro generalizado: 1.2970517815262803e-06
